# Saffman-Taylor instability

Working in the frame of reference moving at the non-dimensionalised unit velocity $\textbf{e}_U$ of the injected fluid.

initial-boundary value problem on $(\textbf{x}, t)\in\Omega\times[0,\infty)$

$$\phi\frac{\partial c}{\partial t} + \textbf{u}\cdot\nabla c = \frac{1}{Pe}\nabla\cdot(\mathsf{D}\cdot\nabla c)$$

$$\nabla\cdot\textbf{u}=0$$

$$\textbf{u}=-\frac{\mathsf{K}}{\mu}\cdot(\nabla p + \mu\mathsf{K}^{-1}\cdot\textbf{e}_U)$$

mapping to porous convection equations defined in `demo/py/porous_convection.py`

$$
\begin{Bmatrix}
\phi \\
Ra \\ 
\mathsf{D} \\
\mathsf{K} \\
\mu \\
\rho \\ 
\textbf{e}_g 
\end{Bmatrix}
\mapsto
\begin{Bmatrix}
\phi \\
Pe \\ 
\mathsf{D} \\
\mathsf{K} \\
\mu \\
\mu \\
\mathsf{K}^{-1}\cdot\textbf{e}_U
\end{Bmatrix}
$$

## Example: $d=2$ rectangle

$$\Omega = [-L_x/2, L_x/2] \times [0, L_y]$$

$$\textbf{n}\cdot\textbf{u}\vert_{x=-L_x/2,y}=\textbf{n}\cdot\textbf{u}\vert_{x=L_x/2,y}$$

$$\textbf{n}\cdot\textbf{u}\vert_{x,y=0}=\textbf{n}\cdot\textbf{u}\vert_{x,y=L_y}=0$$

$$c_0(x,y)=\frac{1}{2}\left(1+\text{erf}\left(-\frac{x}{\epsilon L_x}\right)\right)+\mathcal{N}(x,y)$$

$$c(x=-L_x/2,y)=c(x=L_x/2,y)$$

$$\frac{\partial c}{\partial y}\bigg\vert_{x,y=0}=\frac{\partial c}{\partial y}\bigg\vert_{x,y=L_y}=0$$

$$
\begin{align*}
\phi &= 1 \\
\mathsf{D} &= \mathsf{I} \\ 
\mathsf{K} &= \mathsf{I}\\ 
\mu(c) &= \exp(-R c) \\
\end{align*}
$$

$$\textbf{e}_U=\textbf{e}_x$$

In [ ]:
import numpy as np
import scipy.special as sp
from typing import Callable
from ufl import cos, sin, as_tensor
from lucifex.mesh import rectangle_mesh, mesh_boundary
from lucifex.fdm import FiniteDifference, AB2, CN
from lucifex.fem import LUCiFExConstant as Constant
from lucifex.solver import BoundaryConditions, OptionsPETSc
from lucifex.sim import Simulation, integrate, configure_simulation
from lucifex.utils import CellType, SpatialPerturbation, cubic_noise
from lucifex.viz import plot_colormap, create_animation
from lucifex.io import write

from py.porous_convection import porous_convection_simulation


def saffman_taylor_2d(
    Lx: float = 2.0,
    Ly: float = 1.0,
    Nx: int = 100,
    Ny: int = 100,
    cell: str = CellType.QUADRILATERAL,
    Pe: float = 5e2,
    Rratio: float = 10,
    erf_eps: float = 1e-2,
    c_eps: float = 1e-6,
    c_freq: tuple[int, int] = (8, 8),
    c_seed: tuple[int, int] = (1234, 5678),
):
    
    mesh = rectangle_mesh(
        (-0.5 * Lx, 0.5 * Lx), 
        (-0.5 * Ly, 0.5 * Ly), 
        Nx, 
        Ny, 
        name, 
        cell,
    )
    boundary = mesh_boundary(
        mesh,
        {
            "left": lambda x: x[0] + 0.5 * Lx,
            "right": lambda x: x[0] - 0.5 * Lx,
            "lower": lambda x: x[1] + 0.5 * Ly,
            "upper": lambda x: x[1] - 0.5 * Ly,
        },
    )

    simulation = porous_convection_simulation(
        egx=1
        egy=None,
    )